In [1]:
import os, sys, email
import numpy as np
import pandas as pd
from nltk import tokenize
import pprint, re

from fileRetrieval import path_list
from fileToDataFrame import fileToDataFrame

In [2]:
# ディレクトリ 内のメールファイルを読み込む

directory_path = "enron/maildir/allen-p/"

file_list, dir_list=path_list(directory_path)

In [3]:
# テキストファイルをデータフレームに格納する

df = fileToDataFrame(file_list)

In [4]:
print(df['Message'][0])

Phillip--To the extent that we can give Chair Hoecker our spin on the reasons 
for the hikes, we would like to.  The Commission is getting calls from 
legislators, DOE, etc. about the prices and is going to have to provide some 
response.  Better if it coincides with Enron's view and is not anti-market.  
We still haven't decided what we will provide.  You definitely will be 
included in that discussion once we get the numbers from accounting.  Thanks.


   
	
	
	From:  Phillip K Allen                           12/12/2000 12:03 PM
	

To: Christi L Nicolay/HOU/ECT@ECT
cc:  

Subject: Talking points about California Gas market

Christy,

 I read these points and they definitely need some touch up.  I don't 
understand why we need to give our commentary on  why prices are so high in 
California.  This subject has already gotten so much press.  

Phillip






---------------------- Forwarded by Phillip K Allen/HOU/ECT on 12/12/2000 
12:01 PM ---------------------------
From: Leslie Lawner

In [18]:
from nltk import tokenize

def toSentences(message_text):
    sentences = tokenize.sent_tokenize(message_text)
    cleaned = []
    for sentence in sentences:
        cleaned_sentence = ' '.join(sentence.split())
        if cleaned_sentence[0].isupper()==True:
            cleaned.append(cleaned_sentence)
        else:
            cleaned[-1] = cleaned[-1] + ' ' + cleaned_sentence
    return cleaned

In [20]:
cleaned_sentences = toSentences(df['Message'][0])

In [21]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.structured_prediction
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz")

In [22]:
st = {} #{sentence:predict_result}
for i,s in enumerate(cleaned_sentences):
    st[i] = predictor.predict(sentence = s)

In [77]:
def argIndexList(predict_result:dict):
    triples = {}
    for i,result in predict_result.items():
        triples.setdefault(i,{})
        for verb in result['verbs']:
            ARG0 = [j for j,tag in enumerate(verb['tags']) if re.match('([BI]-)ARG0',tag)]
            ARG1 = [k for k,tag in enumerate(verb['tags']) if re.match('([BI]-)ARG1',tag)]
            if ARG0 != [] and ARG1 != []:
                ARG0 = ' '.join([result['words'][l] for l in ARG0])
                ARG1 = ' '.join([result['words'][m] for m in ARG1])
                triples[i].setdefault(verb['verb'],(ARG0, ARG1))
            else:
                pass
    return triples

In [78]:
argIndexList(st)

{0: {'like': ('we', 'to')},
 1: {'getting': ('The Commission', 'calls'),
  'provide': ('The Commission', 'some response')},
 2: {},
 3: {'decided': ('We', 'what we will provide'), 'provide': ('we', 'what')},
 4: {'get': ('we', 'the numbers')},
 5: {},
 6: {'read': ('I', 'these points'), 'need': ('they', 'some touch up')},
 7: {'understand': ('I',
   'why we need to give our commentary on why prices are so high in California'),
  'need': ('we',
   'to give our commentary on why prices are so high in California'),
  'give': ('we', 'our commentary on why prices are so high in California')},
 8: {},
 9: {'requested': ('they', 'the gas pricing info')},
 10: {'know': ('me or Christi',
   'if you have any disagreements , additions , whatever')},
 11: {},
 12: {}}